## 3.3.3 Cross-encoder 기반의 RAG 시스템 구현

In [7]:
! pip install sentence_transformers
! pip install chromadb
! pip install llama_index.vector_stores.chroma
! pip install onnxruntime
! pip install "numpy<2,>=1.26.0" 

### 1. Cross Encoder 기반 reranker 클래스

In [1]:
# Step1. 크로스 인코더를 이용한 Reranker 클래스
from sentence_transformers import CrossEncoder
from llama_index.core import Document # 임의의 Document 객체를 만들기 위한 클래스 import
from llama_index.core.schema import NodeWithScore, TextNode

class CrossEncoderReranker:
    # 초기화 메서드
    def __init__(self, top_k:int = 2, cross_encoder:CrossEncoder=None,threshold:float=0.0):
        self.top_k = top_k # 관련성 점수 상위 k개 문서 반환
        self.cross_encoder = cross_encoder # 크로스 인코더 모델
        self.threshold = threshold # 관련성 점수 기준값
    
    # query와 문장을 pair로 입력받아 sorting
    def rerank(self, query:str, documents:list[Document]):
        # cross encoder 사용을 위한 쿼리&문서 pair 만들기
        pairs = [[query, doc.text] for doc in documents]
        # cross encoder rerank 수행
        scores = self.cross_encoder.predict(pairs, convert_to_numpy=False) # 
        # score 기준 내림차순 정렬
        scored_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
        # score 상위 k개 반환
        scored_docs = scored_docs[:self.top_k]
        final_docs = []
        for doc, score in scored_docs:
            if score > self.threshold: # 기준값이 넘는 경우에만
                final_docs.append((doc,score))
        return final_docs

c:\Users\USER\OneDrive\Desktop\Project\llamaindex_practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 샘플 문서 리스트
documents = [
    Document(text="인공지능의 발전과 미래 전망에 대한 연구"),
    Document(text="머신러닝 알고리즘의 성능 평가 방법론"),
    Document(text="딥러닝 모델의 학습 최적화 기법"),
    Document(text="교육 분야에서 인공지능의 활용 방안 연구"),
    Document(text="검색 증강 생성 시스템 연구 동향")
]

In [3]:
path = r"./reranker_model/bge-reranker-v2-m3" # 허깅페이스에서 다운받은 bge-reranker 폴더 경로 기재
cross_encoder = CrossEncoder(path) # 크로스 인코더 객체 생성

query = "인공지능의 미래는 어떨까요?"
reranker = CrossEncoderReranker(cross_encoder=cross_encoder) # 크로스 인코더 리랭커 객체 생성
reranked_docs = reranker.rerank(query=query, documents=documents)

In [4]:
# 관련성 점수 상위 n개 순서대로 출력
for idx, (node, score) in enumerate(reranked_docs, start=1):
    print(f"{idx}. {node.text}, score : {score}")

1. 인공지능의 발전과 미래 전망에 대한 연구, score : 0.9195769429206848
2. 교육 분야에서 인공지능의 활용 방안 연구, score : 2.6199459171039052e-05


### 2. 필요한 문서 로드 

In [5]:
### Step4. 데이터 로드
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('./data').load_data()
print(f'문서의 총 개수 : {len(documents)}')

문서의 총 개수 : 83


### 3. Hybrid Search 구현(Embedding+BM25)

In [6]:
from llama_index.core.schema import NodeWithScore
class HybridSearch():
    def __init__(self, bm25_retriever, semantic_retriever,
                 bm25_weight=0.5, semantic_weight=0.5):
        self.bm25_retriever = bm25_retriever # bm25 검색기 객체
        self.semantic_retriever = semantic_retriever # 밀집 검색 객체
        self.bm25_weight = bm25_weight # bm25 가중치
        self.semantic_weight = semantic_weight # 밀집 검색 가중치

    def _get_bm25_retrieve(self, query:str)->list:
        result = self.bm25_retriever.retrieve(query)
        # (node 객체, 가중치 반영 점수)의 튜플 리스트로 반환
        score = [(node.node, node.score * self.bm25_weight) for node in result]
        return score

    def _get_semantic_retrieve(self, query:str)->list:
        result = self.semantic_retriever.retrieve(query)
        # (node 객체, 가중치 반영 점수)의 튜플 리스트로 반환
        score = [(node.node, node.score * self.semantic_weight) for node in result]
        return score

    # 혼합 검색 node 반환
    def retrieve(self, query:str)->list[NodeWithScore]:
        bm25_result = self._get_bm25_retrieve(query=query)
        semantic_result = self._get_semantic_retrieve(query=query)
        
        # key: node 객체, value: 최종 점수
        combined_scores = {}

        # BM25 점수 반영
        for node, score in bm25_result:
            combined_scores[node.text] = {'node':node, 'score':score}
        
        # Semantic 점수 반영
        for node, score in semantic_result:
            if node.text in combined_scores: # 이미 존재하는 청크인 경우
                
                combined_scores[node.text]['score'] += score
            else: # 새로운 청크인 경우
                combined_scores[node.text] = {'node':node, 'score':score}
        
        # NodeWithScore 형태 리스트로 변환
        final_results = []
        for _, node_info in combined_scores.items():
            if node_info['score'] == 0: continue # 점수가 0점인 경우 제외
            final_results.append(NodeWithScore(node=node_info['node'], score=node_info['score']))
        return final_results

### 4. 혼합 검색 사용을 위한 각 검색기 객체 생성

In [15]:
%pip install konlpy
%pip install llama-index-retrievers-bm25

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
###Step4-1. LLM 및 Embedding Model 설정
## API KEY 설정
from dotenv import load_dotenv
load_dotenv()

## 모델 설정
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)  # 모델명은 예시
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [8]:
###Step4-2 희소 검색을 위한 형태소 분석기 함수 작성
from konlpy.tag import Okt
okt = Okt() # Okt 형태소 분석기 초기화
def tokenize_korean_text(text): # 문서 내용 토큰화 함수 정의
    return okt.morphs(text)  # 한국어 형태소 기반 토큰화

In [9]:
### Step4-3. text splitter 설정
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [10]:
### Step4-4. Keyword( BM25 ) Index 생성
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.indices.keyword_table import KeywordTableIndex
keyword_index = KeywordTableIndex.from_documents(
    documents=documents,
    text_splitter=splitter,
    extract_keyword=tokenize_korean_text,
    show_progress=True,
)

# keyword index 생성
keyword_index.storage_context.persist(persist_dir="./index/ch03_keyword_index_storage")

resource module not available on Windows


Extracting keywords from nodes: 100%|██████████| 156/156 [03:45<00:00,  1.44s/it]


In [48]:
###Step4-3 임베딩 모델을 사용 한  VectorStoreIndex 생성
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex.from_documents(
    documents=documents,
    text_splitter=splitter,
    embed_model=embedding_model,
    show_progress=True,
)

vector_index.storage_context.persist(persist_dir="./index/ch03_vector_index_storage")

In [12]:
### 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context1 = StorageContext.from_defaults(persist_dir="./index/ch03_keyword_index_storage") 
storage_context2 = StorageContext.from_defaults(persist_dir="./index/ch03_vector_index_storage") 
keyword_index = load_index_from_storage(storage_context1) # BM25 검색 인덱스 로드
vector_index = load_index_from_storage(storage_context2) # 밀집 검색 인덱스 로드

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_keyword_index_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_keyword_index_storage\index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_vector_index_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_vector_index_storage\index_store.json.


2025-09-30 21:57:10,502 - INFO - Loading all indices.
2025-09-30 21:57:10,661 - INFO - Loading all indices.


In [13]:
## bm25 retriever 객체 생성(keyword_index 연결)
from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(
    index=keyword_index,
    similarity_top_k=5,
)

# dense retriever 객체 생성(vector_index 연결)
semantic_retriever = vector_index.as_retriever(similarity_top_k=5)

2025-09-30 21:57:12,237 - DEBUG - Building index from IDs objects


In [14]:
query='가계부실위험지수에 대해 알고 싶어요.'
# 혼합 검색 클래스 객체 생성
hybrid = HybridSearch(bm25_retriever=bm25_retriever,
                      semantic_retriever=semantic_retriever)
combined_score = hybrid.retrieve(query=query)

2025-09-30 21:57:14,969 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [15]:
# rufrhk 결과 출력
for node_score_obj in combined_score:
    print(node_score_obj)


Node ID: 3e0c3820-e4f7-4aea-9a9d-82e3587405f0
Text: 무역금융지원  프로그램은 수출금융 지원을 위해 무역금융 취급실적에 대해 지원한다. 영세자영업자지 원 프로그램은
영세자영업자 전환대출실적에 대해 지원한다. 지방중소기업지원 프로그 램은 지역 경제사정 등에 부합하는 지방중소기업 대상 대출에
대해 지원한다. 중소기업 대출안정화 프로그램은 중소기업 신용의 변동성 완화 등을 위해 필요시 운영한다. 또한,  신규지원이
종료된 설비투자지원 프로그램 잔액에 대해서도 지원한다. 지원조건을 보면,  대출금리는 프로그램별로 연 0.50~0.75%이다.
대출기간은 1개월이며, 월중 취급실적  변동을 반영하기 위해 월단위로 갱신한다. 대출금액은 은행별 취급실적에 비례한다 .
지원방식을 ...
Score:  1.068

Node ID: 90da87fa-9acb-4dfa-bc53-d0a01c88724f
Text: 63 ㄱ  금융제도 금융거래에 관한 체계와 규범을 총칭하는 개념으로 금융시장, 금융기관, 금융기반구조 (infra-
structure)로 구분된다. 금융시장은 자금의 수요자와 공급자간에 금융거래가 조직 적으로 이루어지는 장소로서 정보시스템 등
추상적 공간을 포함하는 개념이다. 금융시장 은 은행 등 금융중개기관을 통하여 예금, 대출 등의 형태로 자금이전이 이루어지는
간접금융시장과 주식, 채권 등 증권을 통해 자금의 수요자와 공급자간에 직접적인 자금이 전이 이루어지는 직접금융시장으로
구분된다. 금융기관은 자금의 공급자와 수요자간에  거래를 성립시켜 주는 것을 목적으로 하는 사업체로서 우리나라의 경우 은행,
비은행  예...
Score:  1.030

Node ID: 78b0eb85-b480-4a49-8f4a-92d008067cd5
Text: 이는 특히 은행･보험･수출금융 등 금융업무절차나 자금세탁방지 (AML; Anti-Money Laundering)
규제에서 자주 거론된다. 이 절차의 목적은 주로 은행이  자금세탁행위 등의 범죄 요소로 악

### 5. 커스텀 검색기 구현

In [16]:
### Step5. Rerank 적용한 BM25+Custom Retriever 클래스 생성 
# Base query engine import
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.core import QueryBundle

# BM25 -> 크로스인코더 Rerank를 수행하는 Custom Retriever 구현
class CustomRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker:CrossEncoder):
        super().__init__()
        """
        :param base_retriever: 1st ranking 
        :param reranker: Cross Encoder 기반 리랭크 검색기
        """
        self.base_retriever = base_retriever 
        self.reranker = reranker
    # retrieve 함수 customizing
    def _retrieve(self, query_input:str)->list[Document]:
        # 1) QueryBundle이면 내부 query_str를 꺼냄
        if isinstance(query_input, QueryBundle): query = query_input.query_str
        else: query = query_input  # 그냥 문자열 혹은 그 외 케이스
            
        # 1) Base Retriever - 혼합검색
        try: initial_docs = self.base_retriever.retrieve(query)
        except: print('### Base Retrieve Error')
        
        # 2) rerank
        reranked_docs_with_score = self.reranker.rerank(query, initial_docs)
        
        # 3) (Document, score) 튜플에서 Document만 추출
        #    Document + score가 모두 필요하다면 QueryEngine 쪽 custom 로직 필요
        final_docs = [doc_score_tuple[0] for doc_score_tuple in reranked_docs_with_score]
        return final_docs

### 6. 혼합검색 + 크로스 인코더 리랭크 검색기 구현

In [17]:
### step6-1. Hybrid search
# hybrid retriever 
hybrid_retriever = HybridSearch(bm25_retriever=bm25_retriever,
                                semantic_retriever=semantic_retriever,
                                bm25_weight=0.5,
                                semantic_weight=0.5)

### Step6-2. Cross Encoder Rerank
path = r"./reranker_model/bge-reranker-v2-m3"
cross_encoder = CrossEncoder(path) # 크로스 인코더 객체 생성
cross_encoder_rerank = CrossEncoderReranker(top_k=2, cross_encoder=cross_encoder)

### Step6-3. 커스텀 검색기 작성
custom_retriever = CustomRetriever(base_retriever=hybrid_retriever,
                                   reranker=cross_encoder_rerank)

2025-09-30 21:57:22,964 - INFO - Use pytorch device: cpu


In [18]:
'''
CustomRetriever 내부에서 _retrieve()메서드에 대한 내용을 작성했지만
실제 호출시에는 retrieve()를 사용함.
라마인덱스 BaseRetirever의 retrieve() 메서드 내부에서 _retrieve()를 호출하게끔 되어있음.
따라서, 사용자는 retrieve()를 사용하면 원하는 결과를 얻을 수 잇음.
(_retrieve()를 호출해도 결과는 같지만 호환성 문제를 방지하기 위함)
'''
query='가계부실위험지수에 대해 알려주세요'
result = custom_retriever.retrieve(query)

2025-09-30 21:57:26,012 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [01:27<00:00, 87.54s/it]


In [19]:
for one in result:
    print(one, type(one))

Node ID: 83b53498-dba3-4f12-8dba-7e3d8e9f43b6
Text: 1 ㄱ  ㄱ 가계부실위험지수(HDRI) 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여
가계부채의  부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상 환비율(DSR; Debt
Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt  To Asset Ratio)을 결합하여
산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가  각각 40%, 100%일 때 100의 값을 갖도록 설정되어
있으며, 동 지수가 100을 초과하는  가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한
‘고위험가구’,  자산 측면에...
Score:  0.410
 <class 'llama_index.core.schema.NodeWithScore'>
Node ID: f6596d07-1552-4510-9085-8cca100bc3f8
Text: 60 경제금융용어 700선 당국의 주요 관심사가 되었다. 2007년 미국의 서브프라임 모기지 사태와 2008년 9
월  리먼브라더스 파산으로 촉발된 글로벌 금융위기는 세계적인 금융불안과 실물경제 침체 라는 전례가 드문 충격을 가져오면서 각국
정책당국과 시장참가자에게 금융안정의 중요 성과 정책수단의 개발 필요성을 재인식하는 계기가 되었다. 2011년 9월
｢한국은행법｣  개정으로 한국은행은 금융안정 책무를 명시적으로 부여받았다 .  연관검색어 : 시스템 리스크
금융안정위원회(FSB) 기존 금융안정포럼(FSF)의 국제금융시장 안정 기능을 보다 강화하기 위하여 동 포럼의  참여 대상,
책무, 권한 등을 확대 ･개편하여 ...
Score:  0.409
 <class 'llama_index.core.schema.NodeWithScore'>


### 7. 쿼리 엔진 및 응답 생성

In [20]:
### Step7. query engine 생성 및 응답 확인
# Custom Retriever를 사용하기 위해선 RetrieverQueryEngine을 사용해야 함
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever=custom_retriever,) # query engine 구현

In [ ]:
query='가계부실위험지수에 대해 알려주세요'
print(f"Query : {query}")
response = query_engine.query(query)

Query : 가계부실위험지수에 대해 알려주세요


2025-09-30 21:59:25,195 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
print(f"Query : {query}")
print(f"Answer : {response}")

Query : 가계부실위험지수에 대해 알려주세요
Answer : 가계부실위험지수(HDRI)는 가구의 소득 흐름과 금융 및 실물 자산을 종합적으로 고려하여 가계부채의 부실 위험을 평가하는 지표입니다. 이 지수는 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상환비율(DSR)과 자산 측면에서 평가하는 부채/자산비율(DTA)을 결합하여 산출합니다. 가계부실위험지수는 DSR과 DTA가 각각 40%와 100%일 때 100의 값을 갖도록 설정되어 있으며, 이 지수가 100을 초과하는 가구는 '위험가구'로 분류됩니다. 위험가구는 소득 및 자산 측면에서 모두 취약한 '고위험가구', 자산 측면에서 취약한 '고DTA가구', 소득 측면에서 취약한 '고DSR가구'로 구분할 수 있습니다. 이 지수는 가구의 채무상환능력의 취약성을 평가하는 것이며, 당장 채무상환 불이행을 의미하지는 않습니다.


In [31]:
for v in response.source_nodes:
    print(v)

Node ID: 83b53498-dba3-4f12-8dba-7e3d8e9f43b6
Text: 1 ㄱ  ㄱ 가계부실위험지수(HDRI) 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여
가계부채의  부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상 환비율(DSR; Debt
Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt  To Asset Ratio)을 결합하여
산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가  각각 40%, 100%일 때 100의 값을 갖도록 설정되어
있으며, 동 지수가 100을 초과하는  가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한
‘고위험가구’,  자산 측면에...
Score:  0.410

Node ID: f6596d07-1552-4510-9085-8cca100bc3f8
Text: 60 경제금융용어 700선 당국의 주요 관심사가 되었다. 2007년 미국의 서브프라임 모기지 사태와 2008년 9
월  리먼브라더스 파산으로 촉발된 글로벌 금융위기는 세계적인 금융불안과 실물경제 침체 라는 전례가 드문 충격을 가져오면서 각국
정책당국과 시장참가자에게 금융안정의 중요 성과 정책수단의 개발 필요성을 재인식하는 계기가 되었다. 2011년 9월
｢한국은행법｣  개정으로 한국은행은 금융안정 책무를 명시적으로 부여받았다 .  연관검색어 : 시스템 리스크
금융안정위원회(FSB) 기존 금융안정포럼(FSF)의 국제금융시장 안정 기능을 보다 강화하기 위하여 동 포럼의  참여 대상,
책무, 권한 등을 확대 ･개편하여 ...
Score:  0.409

